#Consignas TP3
En este momento ya definimos la variable target, ICA.
Ahora debemos responder...
Qué modelos de regresión existen y podemos utilizar?
Qué métricas existen? Cuál es conveniente utilizar? Cómo funciona GridSearch y RamdonSearch?


Una vez definida la métrica a comparar se generara un primer modelo baseline que nos servirá de punto de comparación con otros modelos. Para realizar la comparación tendremos que tener en cuanta cuáles son las métricas más apropiadas para optimizar y se realizaran ajustes por hiperparámetros en los diferentes modelos.

Concluiremos eligiendo el modelo más adecuado para la predicción propuesta.

# Importar Librerias

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


# Carga de los dataset

In [2]:
df_21_23 = pd.read_csv('https://raw.githubusercontent.com/MaricelSantos/Mentoria-Diplodatos-2024/main/Conexiones_Transparentes_21-23.csv')
df_22_sys=  pd.read_csv('https://raw.githubusercontent.com/MaricelSantos/Mentoria-Diplodatos-2024/main/Conexiones_Transparentes.csv')

In [4]:
df_21_23

,orden,sitios,codigo,fecha,año,campaña,tem_agua,tem_aire,od,ph,...,Poblacion_partido,Personas_con_cloacas,Actividad_principal,"Agricultura, ganadería, caza y silvicultura",Pesca explotación de criaderos de peces y granjas piscícolas y servicios conexos,Explotación de minas y canteras,Industria Manufacturera,"Electricidad, gas y agua",Construcción,Servicios
0,1.0,Canal Villanueva y Río Luján,TI001,23/2/2022,2022,Verano,24.5,23.3,5.3,6.56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,Canal Villanueva y Río Luján,TI001,4/5/2022,2022,otoño,19.6,16,7.73,7.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,Canal Villanueva y Río Luján,TI001,23/8/2022,2022,invierno,13.8,14,8.61,7.06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,Canal Villanueva y Río Luján,TI001,31/10/2022,2022,Primavera,21.5,15,8.3,"7,8",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,Río Lujan y Arroyo Caraguatá,TI006,23/2/2022,2022,Verano,25.4,23.3,2.25,6.56,...,446.949,144.834,"""La industria metalúrgica y la actividad mader...",4.0,4.0,4.0,4.0,5.0,3.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,NaN,NaN,CA CU02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.121.707,3.049.672,"""servicios inmobiliarios, financieros y comercio""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
517,NaN,NaN,CA CU03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.121.707,3.049.672,"""servicios inmobiliarios, financieros y comercio""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,NaN,NaN,CA CU04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.121.707,3.049.672,"""servicios inmobiliarios, financieros y comercio""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
519,NaN,NaN,CA CU05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.121.707,3.049.672,"""servicios inmobiliarios, financieros y comercio""",NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Eliminacion de filas y columnas

Recuerden que pueden comparar un dataset que tenga ciertas columnas con otro que no las tenga para ver cual es la influencia de esas variables.

# Separación del dataset
Se los muestro de forma generica. Pero como hablamos nuestra variable target seria ICA el indice de calidad de agua. RECUERDEN HACER ESTE PASO ANTES DE LAS TRANSFORMACIONES.Si tiene más de un dataset van a tener una separación para cada uno de ellos.

In [ ]:
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Definición de Columnas y Transformaciones
Es necesario identificar las columnas numéricas y categóricas porque vamos a tener diferentes transformaciones según cada tipo de columna.



In [ ]:
numeric_features = ['num_col1', 'num_col2']
categorical_features = ['cat_col1', 'cat_col2']


# Curación y Creación de Múltiples Versiones del Dataset

Podemos comparar dataset donde fueron removidos outliers o fueron removidad columnas. Pueden probar con todos los que deseen. Abajo el codigo para quitar outliers. Recuerden cada uno de ellos va a tener su división en train y en test.

In [ ]:
def remove_outliers(df, columns):
    for col in columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        df = df[(df[col] >= (q1 - 1.5 * iqr)) & (df[col] <= (q3 + 1.5 * iqr))]
    return df

#Generamos el df sin outliers
df_no_outliers = remove_outliers(df, numeric_features)

#Separamos el df sin outliers
X_train_no_outliers, X_test_no_outliers, y_train_no_outliers, y_test_no_outliers = train_test_split(
    df_no_outliers.drop('target', axis=1), df_no_outliers['target'], test_size=0.2, random_state=42)

Ejemplo de una transformacion/curacion sobre númericas de un dataset

In [ ]:
#Defino el transformados que se va aplicar
scaler = StandardScaler()

#Aplico transformador a las numericas del train del dataset origical

X_train_normal = scaler.fit_transform(X_train[numeric_features])

# El paso anterior devuelve un array por lo tanto es necesario volver a obtener panda df
df_normalizado_train = pd.DataFrame(X_train_normal, columns=X_train[numeric_features].columns)

# En este paso se concatena lo obtenido con lo original o con lo que venga trabajando
# por ejemplo concatenar lo aplicado a las numericas con lo aplicado a las categoricas
X_train_normal_hotencoding = pd.concat([df_normalizado_train, df_hotencoding_train], axis=1)

#Repito los pasos con el test pero con el cuidado de que aquí no utilizo fit_transform
#solo transform porque se trata de test

X_test_normal = scaler.transform(X_test[numeric_features])
df_normalizado_test = pd.DataFrame(X_test_normal, columns=X_test[numeric_features].columns)
X_test_normal_hotencoding = pd.concat([df_normalizado_test, df_hotencoding_test], axis=1)


Una vez finalizada la curación pasamos a probar varios modelos a la vez. Como lo hacemos? usando un ciclo for. Supongamos que guardamos el resultados de train en la variable X_train_final y los resultados de test en X_test_final. A su vez tenemos las variables target separada para cada grupo y_train y y_tesT

#Modelos
Vamos a ver dos formas de hacerlo. La primera es la menos compleja. La segunda es más compleja pero es lo que se espera que logren para poder darle la vuelta de rosca que necesita.
En la primera forma no hay ajuste de hiperparametros.
En la segunda se usa randomsearch para ajustar hiperparametros.

In [ ]:
# Defino una lista de los modelos que voy a testear.
#La idea es que probemos varios para comparar la metrica seleccionada


clfs =  [DecisionTreeClassifier(random_state=8),
        RandomForestClassifier(random_state=8),
        LogisticRegression(random_state=8),
        MLPClassifier(random_state=8),
        XGBClassifier(random_state=8),
        GaussianNB(),
        LinearSVC(random_state=8)]

# Elijo nombres lindos para el print
names = ['Arbol de decisión',
        'Random Forest',
        'Regresión Logística',
        'Perceptrón multicapa',
        'XGBoost',
        'Naive Bayes',
        'SVM']

trained_models = []
for clf,name in zip(clfs,names):
    print(name)
    clf.fit(x_train_final, y_train)
    train_predictions = clf.predict(x_train_final)
    accuracy = accuracy_score(y_train, train_predictions) #Aqui va la metrica que elijan, accuracy no es correcto para regresiones, es para clasificaciones
    print(f"Accuracy train {name}: %.2f%%" % (accuracy * 100.0))

    test_predictions = clf.predict(x_test_final)
    accuracy = accuracy_score(y_test, test_predictions)
    print("Accuracy test {name}: %.2f%%" % (accuracy * 100.0)) #Aqui va la metrica que elijan, accuracy no es correcto para regresiones, es para clasificaciones
# Guardo la informacion correspondiente al modelo y los resultados en una lista de tuplas
    trained_models.append((clf, (accuracy*100)))

#Busqueda de hiperparametros
Computacionalmente es más costoso. Es muy importante almacenar la variable tiempo porque también es una métrica a comparar. Si dos modelos me dan el mismo r2 pero uno de ellos me da menor tiempo de computo, voy a elegir este último.

In [ ]:
# Definimos los modelos y sus respectivos hiperparámetros en un diccionario. LLaver: nombre del modelo:
models = {
    'Lasso': {
        'model': Lasso(),
        'params': {
            'alpha': [0.001, 0.003, 0.01]
        }
    },
    'ElasticNet': {
        'model': ElasticNet(),
        'params': {
            'alpha': [0.001, 0.003, 0.01],
            'l1_ratio': [0.3, 0.5, 1.0]
        }
    },
    'Decision Tree': {
        'model': DecisionTreeRegressor(),
        'params': {
            'max_depth': [8, 15, 30],
            'min_samples_split': [20, 50, 80],
            'min_samples_leaf': [1, 5, 10]
        }
    },
        'RandomForest': {
        'model': RandomForestRegressor(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [8, 15, 30],
            'min_samples_split': [20, 50, 80],
            'min_samples_leaf': [1, 5, 10],
            'random_state': [42]
        }
    },
        'GradientBoostingRegressor': {
        'model': GradientBoostingRegressor(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [8, 15, 30],
            'min_samples_split': [20, 50, 100],
            'min_samples_leaf': [1, 5, 10],
            'learning_rate': [0.05, 0.1, 0.2],
            'random_state': [42]
        }
    },
        'XGBoost': {
        'model': XGBRegressor(),
        'params': {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.001],
            'max_depth': [5, 10, 15],
            'min_child_weight': [1, 2, 3],
            'gamma': [0, 0.1, 0.2],
        }
    }
}


In [ ]:
results = {}  # Para almacenar los resultados

for model_name, model_config in models.items():
    # Registra el tiempo de inicio.
    start_time = time.time()

    # Define los hiperparámetros que deseas ajustar
    param_grid = model_config['params']

    # Configura RandomizedSearchCV para el modelo y los hiperparámetros
    random_search = RandomizedSearchCV(model_config['model'], param_grid,
                                     n_iter=15, scoring='r2', cv=5,
                                     random_state=42)

    # Realiza la búsqueda de hiperparámetros en los datos de entrenamiento
    random_search.fit(X_train_final, y_train)

    # Registra el tiempo de finalización
    end_time = time.time()

    # Calcula el tiempo transcurrido en segundos
    elapsed_time = end_time - start_time

    # Predecir en el conjunto de prueba
    best_estimator = random_search.best_estimator_
    y_pred_test = best_estimator.predict(X_test_final)
    y_pred_train = best_estimator.predict(X_train_final)

   # Calcular la precisión del modelo
    mse_test = mean_squared_error(y_test, y_pred_test)
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)
    print(f"mse_train: {mse_train:.2f},mse_test: {mse_test:.2f} , r2: {r2_test:.2f} ")

    # Almacena los resultados

    results[(model_name)] = {
        'best_params': random_search.best_params_,
        'r2_train': random_search.best_score_,
        'r2_test': r2_test,
        'mse_train': mse_train,
        'mse_test': mse_test,
        'elapse_time':  elapsed_time
    }

results_df = pd.DataFrame(results)
results_df.to_csv('results.csv', index=False)
files.download('results.csv')


Ambos bloques (con y sin busqueda de hiperparametros) estan hechos para un dataset. Si quiero comparar 2 dataset tengo que definir un bloque similar pero con las variables correspondientes al segundo (train/test). Luego puedo comparar metricas guardadas.
En la busqueda de hiperparametros lo mejor es guardar los resultados en un csv porque van a ver que demora mucho en resolver, entonces no pueden estar corriendo cada vez que lo quieran ver.
La intención es que las metricas guardadas sean luego graficadas para mostrar comparativamente los resultados de cada modelo y dataset trabajado. Recuerden que la entrega es un VIDEO SIN CODIGO.

#Importancia de las variables
Para la mayoría de los metodos de sklear tiene el metodo .features_importance_ En LinearRegression y LogisticRegression
puden ver los coeficientes del modelo con el metodo .coef_
Estos metodos le van a dar de forma ordenada justamente la importancia o el peso de cada variable en el modelo. Usenlos para sacar conclusiones. Investiguen segun los modelos que usan que métodos le dan esa información.